In [2]:
# import of adj_mat
# import of nodes_cluster
# import of news_data
# import of crypto_data

# Import section

## adj_mat

## nodes_cluster

## news_data

## crypto_data

# Merging section (probably needed)

# Model definition (probably needed)

## max. cluster strength

In [ ]:
# cluster strength

## input:
### date
### cluster annotation per node, per day
## throughput1:
### number of neighbors inside cluster per node, per day
### number of total neighbors per node, per day (degree per node, per day)
## throughput2:
### (number of neighbors inside cluster per node, per day) / (degree per node, per day)
## output:
### lowest value of throughput2, per day








## sentiment score

## indicator score

## model payoffs

In [1]:
# model_payoffs:

## input:
### x --> linear combination of scores
### date
## output:
### payoff a --> a = 1/(1+exp(-x)), per day
### payoff b --> b = 1 - a, per day


def model_payoffs(**):
    x = sum(keyword_arguments)
    
    a = 1/(1+exp(-x))
    b = 1 - a
        
    payoffs = []
    
    payoffs.append(a)
    payoffs.append(b)
    
    
    return payoffs

# threshold:

## input:
### payoff for strategy a
### payoff for strategy b
## output:
### threshold value q --> q = b / (a + b)

def threshold(**):
    payoff_a = keyword_arguments[0]
    payoff_b = keyword_arguments[1]
    
    q = payoff_b / (payoff_a + payoff_b)
    
    return q

def model_output(date, q, payoffs):
    #create dictionary with {date: [payoffs]}


    

